MODEL 1:
Text to data Conversion:
The following code will be using doc2vec as it is more representative of the text with the context, as compared to vectorizer which simply counts the number of times a word is found in the data.
Algorithm:
The following shall use Stacking to predict the classes for the books.

Packages:

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Pre-Processing:

In [31]:
book_training_file = "project_data_files/book_rating_train.csv"
book_testing_file = "project_data_files/book_rating_test.csv"

train_names_file = "project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv"
train_authors_file = "project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv"
train_desc_file = "project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv"

test_names_file = "project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv"
test_authors_file = "project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv"
test_desc_file = "project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv"

train_data = pd.read_csv(book_training_file)
test_data = pd.read_csv(book_testing_file)

word_training_files = [train_names_file, train_authors_file, train_desc_file]
word_testing_files = [test_names_file, test_authors_file, test_desc_file]

word_train_data = [pd.read_csv(filename, index_col = False, delimiter = ',', header=None) for filename in word_training_files]
word_test_data = [pd.read_csv(filename, index_col = False, delimiter = ',', header=None) for filename in word_testing_files]

Baseline: 0R

In [36]:
def zero_R(labels):
    ratings, rating_counts = np.unique(labels, return_counts=True)
    num_labels = len(labels)

    max_prob = (0, labels[0])
    for i in range(len(ratings)):
        prob = rating_counts[i] / num_labels
        if prob > max_prob[0]:
            max_prob = (prob, ratings[i])
    return max_prob[1]


array([3., 4., 5.])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train_data[train_data.columns[:-1]], train_data[train_data.columns[-1]], test_size=0.2, random_state=1169800)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((18450, 9), (18450,), (4613, 9), (4613,))